In [1]:
# ref: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False,
                                       download=True, transform=transform)

# split train set / valid set
validation_ratio=0.1
random_seed= 17

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          sampler=train_sampler, num_workers=2)
validloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          sampler=valid_sampler, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(32*32*3, 256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net = Net().to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=0.0001)

exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

Files already downloaded and verified
Files already downloaded and verified
cuda:0


In [2]:
best_valid_acc=0
for epoch in range(50):  # loop over the dataset multiple times
    
    net.train()
    ### learnint rate scheduling ###
    exp_lr_scheduler.step()
    
    running_loss = 0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # for train acc
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    

    ### train loss/acc ###
    print('[%d] loss: %.3f, train_acc: %.3f %%, lr:%f' %
          (epoch + 1, running_loss / 50000, 100.*correct/total, exp_lr_scheduler.get_lr()[0]))
    
    net.eval()
    ### valid acc ###
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    valid_acc= 100.*correct/total
    if best_valid_acc<valid_acc:
        best_valid_acc= valid_acc
        torch.save(net.state_dict(), "./save_best.pth") #save
    print('[%d] valid_acc: %.3f %%, best: %.3f %%' %(epoch + 1, valid_acc, best_valid_acc))
    
    


print('Finished Training')

[1] loss: 0.012, train_acc: 38.776 %, lr:0.010000
[1] valid_acc: 43.280 %, best: 43.280 %
[2] loss: 0.011, train_acc: 44.416 %, lr:0.010000
[2] valid_acc: 43.460 %, best: 43.460 %
[3] loss: 0.011, train_acc: 45.924 %, lr:0.010000
[3] valid_acc: 42.980 %, best: 43.460 %
[4] loss: 0.010, train_acc: 47.022 %, lr:0.010000
[4] valid_acc: 45.960 %, best: 45.960 %
[5] loss: 0.010, train_acc: 47.898 %, lr:0.010000
[5] valid_acc: 46.240 %, best: 46.240 %
[6] loss: 0.010, train_acc: 48.196 %, lr:0.010000
[6] valid_acc: 47.900 %, best: 47.900 %
[7] loss: 0.010, train_acc: 48.729 %, lr:0.010000
[7] valid_acc: 47.920 %, best: 47.920 %
[8] loss: 0.010, train_acc: 49.011 %, lr:0.010000
[8] valid_acc: 46.100 %, best: 47.920 %
[9] loss: 0.010, train_acc: 49.191 %, lr:0.010000
[9] valid_acc: 47.140 %, best: 47.920 %
[10] loss: 0.010, train_acc: 49.376 %, lr:0.010000
[10] valid_acc: 46.860 %, best: 47.920 %
[11] loss: 0.010, train_acc: 49.758 %, lr:0.010000
[11] valid_acc: 48.380 %, best: 48.380 %
[12] l

In [3]:
# load & eval

load_model = Net().to(device)
load_model.load_state_dict(torch.load("./save_best.pth"))

load_model.eval()

### test acc ###
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = load_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc= 100.*correct/total

print('test_acc: %.3f %%' %(test_acc))


test_acc: 54.170 %
